In [1]:
import pandas as pd
import datetime


GESAMTFILE = '/opt/data/optimax_sensing/Gesamtfile_EMA_Baseline_bereinigt_29.01.22_MD.xlsx'

df = pd.read_excel(GESAMTFILE, engine='openpyxl')
df = df[['VP','Date_Time_last_action (MAX) / completed (SEMA3)','Teilnehmer Mattermost']]

#Generate placeholder values for missing columns 
df[['fitbit_id','empatica_id']] = None
df['platform'] = 'infer'
df['label'] = df['VP']
df['end_date'] = None #df['Date_Time_last_action (MAX) / completed (SEMA3)']

#rename and order columns
df = df[['Teilnehmer Mattermost','fitbit_id','empatica_id','VP','platform','label','Date_Time_last_action (MAX) / completed (SEMA3)','end_date']]
df.columns = ['device_id','fitbit_id','empatica_id','pid','platform','label','start_date','end_date']

#drop NAs in pid column 
df = df.dropna(subset=['pid'])
df = df.dropna(subset=['device_id'])

#Convert dtypes
df = df.astype({"label": int, "pid": int})
df['start_date'] = pd.to_datetime(df['start_date'], infer_datetime_format = True)
df['start_date'] = df['start_date'].dt.round('D') #round to day



In [3]:

#group by participant and get first and last date, put in last two columns 
end_dates = df.groupby('pid')['start_date'].max().to_frame()
start_dates = df.groupby('pid')['start_date'].min().to_frame()


df = df.drop_duplicates(subset=['pid']) #delete duplicate rows 

df = df.drop(['start_date','end_date'], axis = 1)

df = df.merge(start_dates, how = 'inner', on = ['pid'])
df = df.merge(end_dates, how = 'inner', on = ['pid'])

#rename columns
df.columns = ['device_id','fitbit_id','empatica_id','pid','platform','label','start_date','end_date']


In [35]:
df


,device_id,fitbit_id,empatica_id,pid,platform,label,start_date,end_date
0,5b34dda0d5487d4258ec82ef,None,None,102,infer,102,2018-06-29,2018-07-13
1,5b50bd01d5487d4258ecbeff,None,None,104,infer,104,2018-07-21,2018-07-22
2,5b476480d5487d4258ecad93,None,None,105,infer,105,2018-07-13,2018-07-25
3,5b864d34cf88d7c89a998aa4,None,None,106,infer,106,2018-09-06,2018-09-13
4,5b68591ccf88d7c89a99280d,None,None,115,infer,115,2018-08-07,2018-08-21
...,...,...,...,...,...,...,...,...
72,5fb43011f785aebd119f0a32,None,None,393,infer,393,2020-11-18,2020-12-02
73,601c471f3800bbe3308580b9,None,None,400,infer,400,2021-02-05,2021-02-19
74,6006babc3800bbe33084d37e,None,None,401,infer,401,2021-01-21,2021-02-03
75,60264f083800bbe33085d550,None,None,405,infer,405,2021-02-13,2021-02-26


In [ ]:
#save

df.to_csv('/opt/data/optimax_sensing/BaselineParticipantsOptimax')